This notebook is the reference for the video lecture on the Linear Regression Code Along. Basically what we do here is examine a dataset with Ecommerce Customer Data for a company's website and mobile app. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("LRCodeAlong").getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv("Ecommerce_Customers.csv", inferSchema=True, header=True)

In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [6]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [7]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [8]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [9]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [10]:
from pyspark.ml.linalg import Vector

In [11]:
from pyspark.ml.feature import VectorAssembler

In [12]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'], outputCol="features")

In [13]:
output = assembler.transform(data)

In [14]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [15]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [16]:
final_data = output.select("features","Yearly Amount Spent")

In [17]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [18]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                340|
|   mean| 499.72234641863076|
| stddev|  80.20304038611654|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [19]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                160|
|   mean| 498.44638341850595|
| stddev|  77.63494764901183|
|    min|  304.1355915788555|
|    max|  712.3963268096637|
+-------+-------------------+



In [22]:
lr = LinearRegression(labelCol='Yearly Amount Spent')
# Create a Linear Regression Model object

In [21]:
lrModel = lr.fit(train_data)
# Fit the model to the data and call this model lrModel

In [23]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.79586682328467,38.31534752861182,0.6532320348768325,61.37462653063605] Intercept: -1056.9121416898317


In [24]:
test_result = lrModel.evaluate(test_data)

In [25]:
test_result.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  10.960136533992227|
|  -2.747533502836575|
|   -4.87937617088329|
| -12.052235177731689|
| 0.22029657406676506|
|  19.173440625502792|
|  0.6962277147774785|
|  -0.777480900202022|
| -3.7801121224391068|
|   7.523275377144216|
|  -0.379752251108016|
|  -6.412393528221969|
|  -3.916957282460828|
|   7.708841830810059|
| -1.7001756254455813|
|-0.27544283348186127|
|   -5.70092341877961|
|  -1.291934900775857|
|   6.214364348283311|
|   5.238689330066336|
+--------------------+
only showing top 20 rows



In [32]:
unlabeled_data = test_data.select('features')
#intersecting result

In [27]:
predictions = lrModel.transform(unlabeled_data)

In [28]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|397.68021453863525|
|[30.8364326747734...| 470.2494339298262|
|[31.0613251567161...| 492.4348342287849|
|[31.0662181616375...|460.98552838540604|
|[31.2606468698795...| 421.1063346828846|
|[31.3123495994443...| 444.4179774024378|
|[31.3895854806643...| 409.3733833452054|
|[31.5761319713222...| 542.0040648895304|
|[31.6253601348306...| 380.1170128793633|
|[31.6548096756927...| 467.7401483504043|
|[31.6610498227460...|416.73810583100885|
|[31.7207699002873...| 545.1873270062449|
|[31.7656188210424...|500.47103891806796|
|[31.8209982016720...| 416.9664391824033|
|[31.8530748017465...|460.98529908779756|
|[31.8627411090001...| 556.5735840075286|
|[31.8745516945853...| 397.9861676650471|
|[31.9120759292006...| 388.8266512064836|
|[31.9480174211613...| 455.7065125446145|
|[31.9764800614612...|325.35575670403387|
+--------------------+------------

In [30]:
print("RMSE: {}".format(test_result.rootMeanSquaredError))
print("MSE: {}".format(test_result.meanSquaredError))

RMSE: 9.653784241728912
MSE: 93.19555018585348
